In [20]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import re



def get_and_parse_url(url):
    result = requests.get(url)
    soup = BeautifulSoup(result.text, 'html.parser')
    return(soup)

In [21]:
main_url = 'https://www.politikensforlag.dk/boeger/c-2?page=1&sort=1&view=list#header'

result = requests.get(main_url)
soup = BeautifulSoup(result.text, 'html.parser')

In [22]:
def get_book_url(url):
    soup = get_and_parse_url(url)
    # remove the index.html part of the base url before returning the results
    return([x.div.a.get('href') for x in soup.findAll("div", class_ = "textblock")])

In [4]:
page_urls = []

i = 1
while i < 117:
    new_page = 'https://www.politikensforlag.dk/boeger/c-2?page=' + str(i) + '&sort=1&view=list' + '.html'
    page_urls.append(new_page)
    i += 1

book_urls = []

for page in pages_urls:
    book_urls.extend(get_book_url(page))

In [5]:
b_urls = booksURLs
for n in range(len(b_urls)):
    if b_urls[n][0] == 'h':
        print(b_urls[n], ' Done!!')
        break
        
    b_urls[n] = "https://www.politikensforlag.dk" + b_urls[n]

b_urls[:5]

['https://www.politikensforlag.dk/mod-pa-livet/t-2/9788740056945',
 'https://www.politikensforlag.dk/i-ruiner/t-2/9788740054859',
 'https://www.politikensforlag.dk/heartstopper-2/t-2/9788740067620',
 'https://www.politikensforlag.dk/heartstopper-1/t-2/9788740067613',
 'https://www.politikensforlag.dk/minecraft-rejsen/t-2/9788740051612']

In [38]:
def get_book_info(book_url):
    
    title, author, price, sideantal, sprog, udgivelse = None, None, None, None, None, None
    
    soup = get_and_parse_url(book_url)
    
    soup = str(soup)
    
    if re.search('Sideantal:[ 0-9]+', soup) != None:
        sideantal = re.search('Sideantal:[ 0-9]+', soup).group()
        sideantal = re.search('[0-9]+', sideantal).group()
        sidenantal = int(sideantal.strip())
    
    if  re.search("'price'		: '[0-9]+", soup) != None:
        price = re.search("'price'		: '[0-9]+", soup).group()
        price = re.search('[0-9]+', price).group()
        price = int(price.strip())
    
    if re.search("Sprog: [a-zA-Z]+", soup) != None:
        sprog = re.search("Sprog: [a-zA-Z]+", soup).group()
        sprog = sprog.split()[1]
    
    if re.search("<title>[\s\S]+ <\/title>", soup) != None:
        head = re.search("<title>[\s\S]+ <\/title>", soup).group()
        head = head.split('|')
        
    if re.search('Udk\.[\s\S]{20}', soup) != None:
        udgivelse = re.search('Udk\.[\s\S]{20}', soup).group()
        udgivelse = udgivelse[5:].split('<')[0]

        title = head[0].split('>')
        title = title[1]

        author = head[1]
        author = author.split()
        author = " ".join(author)
    
    x = [title, author, udgivelse, price, sideantal, sprog]
    return x

In [40]:
book_info = []

i = 0
for url in b_urls:
    i += 1
    row = get_book_info(url)
    book_info.append(row)
    if i%100 == 0:
        print(i)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300


In [41]:
column_names = ["Titel", "Forfatter", "Udgivelesdato", "Pris", "Sideantal", "Sprog"]
data = pd.DataFrame(book_info, columns = column_names)

data

,Titel,Forfatter,Udgivelesdato,Pris,Sideantal,Sprog
0,Mod på livet,Pia Callesen,28. september 2021,250,248,Dansk
1,I ruiner,Morten Pape,19. august 2021,300,448,Dansk
2,Heartstopper 2,Alice Oseman,10. juni 2021,200,400,Dansk
3,Heartstopper 1,Alice Oseman,10. juni 2021,200,400,Dansk
4,Minecraft - Rejsen,Jason Fry,01. juni 2021,130,None,Dansk
...,...,...,...,...,...,...
1376,Sigurd fortæller spidsnæsehistorier,Sigurd Barrett,08. februar 2010,130,108,Dansk
1377,Politiken mod Politiken,Politikens Forlag </title>,08. september 2009,350,576,Dansk
1378,None,None,None,100,None,Dansk
1379,Nynnes dagbog 2,"Henriette Lind, Lotte Thorsen",25. januar 2007,100,288,Dansk


In [ ]:
drenge = pd.read_excel('drengenavne.xls')
drenge = drenge['Ab'].tolist()
piger = pd.read_excel('pigenavne.xls')
piger = piger['Aba'].tolist()

køn = []
for i in data['Forfatter']:
    
    if i is None:
        køn.append(None)
        continue
        
    fornavn = i.split()[0]
    
    if fornavn in drenge and fornavn in piger:
        køn.append(None)
        continue
        
    if fornavn in drenge:
        køn.append("M")
        continue
        
    if fornavn in piger:
        køn.append("K")
        continue
    
    if fornavn not in drenge and fornavn not in piger:
        køn.append(None)
        continue

data['køn'] = køn

In [88]:
dates = data['Udgivelesdato'].tolist()

months = {"januar":"1", "februar":"2", "marts":"3", "april":"4", "maj":"5", "juni":"6", "juli":"7", "august":"8", "september":"9", "oktober":"10", "november":"11", "december":"12"}

for i in range(len(dates)):
    
    if dates[i] == None:
        continue
        
    dato = dates[i].split()
    
    dag = dato[0][:2]
    
    if dag[0] == "0":
        dag = dag[1]
    
    måned = months[dato[1]]
        
    år = dato[2]
    
    dates[i] = måned + "/" + dag + "/" + år

data['Udgivelesdato	'] = dates

In [89]:
data

,Titel,Forfatter,Udgivelesdato,Pris,Sideantal,Sprog,køn,Udgivelsesdato,Udgivelesdato\t
0,Mod på livet,Pia Callesen,28. september 2021,250,248,Dansk,K,9/28/2021,9/28/2021
1,I ruiner,Morten Pape,19. august 2021,300,448,Dansk,M,8/19/2021,8/19/2021
2,Heartstopper 2,Alice Oseman,10. juni 2021,200,400,Dansk,K,6/10/2021,6/10/2021
3,Heartstopper 1,Alice Oseman,10. juni 2021,200,400,Dansk,K,6/10/2021,6/10/2021
4,Minecraft - Rejsen,Jason Fry,01. juni 2021,130,None,Dansk,M,6/1/2021,6/1/2021
...,...,...,...,...,...,...,...,...,...
1376,Sigurd fortæller spidsnæsehistorier,Sigurd Barrett,08. februar 2010,130,108,Dansk,M,2/8/2010,2/8/2010
1377,Politiken mod Politiken,Politikens Forlag </title>,08. september 2009,350,576,Dansk,None,9/8/2009,9/8/2009
1378,None,None,None,100,None,Dansk,None,None,None
1379,Nynnes dagbog 2,"Henriette Lind, Lotte Thorsen",25. januar 2007,100,288,Dansk,K,1/25/2007,1/25/2007


In [72]:
writer = pd.ExcelWriter('JP_POL-books.xlsx')
data.to_excel(writer)

writer.save()
print('DataFrame is written successfully to Excel File.')

DataFrame is written successfully to Excel File.


In [92]:
data2 = data

data2 = data2.drop(columns = ['Udgivelesdato', 'Udgivelesdato\t'])

data2

,Titel,Forfatter,Pris,Sideantal,Sprog,køn,Udgivelsesdato
0,Mod på livet,Pia Callesen,250,248,Dansk,K,9/28/2021
1,I ruiner,Morten Pape,300,448,Dansk,M,8/19/2021
2,Heartstopper 2,Alice Oseman,200,400,Dansk,K,6/10/2021
3,Heartstopper 1,Alice Oseman,200,400,Dansk,K,6/10/2021
4,Minecraft - Rejsen,Jason Fry,130,None,Dansk,M,6/1/2021
...,...,...,...,...,...,...,...
1376,Sigurd fortæller spidsnæsehistorier,Sigurd Barrett,130,108,Dansk,M,2/8/2010
1377,Politiken mod Politiken,Politikens Forlag </title>,350,576,Dansk,None,9/8/2009
1378,None,None,100,None,Dansk,None,None
1379,Nynnes dagbog 2,"Henriette Lind, Lotte Thorsen",100,288,Dansk,K,1/25/2007


In [93]:
writer = pd.ExcelWriter('JP_POL-books2.xlsx')
data2.to_excel(writer)

writer.save()
print('DataFrame is written successfully to Excel File.')

DataFrame is written successfully to Excel File.
